# Notebook to run the gaussian regularized alghorithms 

In [15]:
import ourutils 

ourutils.read_add_paths('paths.txt')
paths = ourutils.read_paths('paths.txt')


import estimators as es 
import run_parallel as run
import numpy as np
import os
import importlib
from utils import *

importlib.reload(ourutils)
importlib.reload(run)

<module 'run_parallel' from 'C:\\Users\\Francesca\\OneDrive\\Documenti\\BrainHack\\covariance-estimators\\run_parallel.py'>

In [17]:
nsamples = 200
nfeatures = 116

X = np.random.randn(nsamples, nfeatures)  # dummy dataset with true covariance I
Xtrain = X[:int(0.8*nsamples)]
Xtest = X[int(0.8*nsamples):]

res = es.fit_PCA(Xtrain, Xtest, ncomponents=10)
# res = es.fit_PCA_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_PCA_Minka(Xtrain, Xtest)
# res = es.fit_ConservativePCA(Xtrain, Xtest, p=10)
# res = es.fit_ConservativePCA_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_FactorAnalysis(Xtrain, Xtest)
# res = es.fit_FactorAnalysis_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_GraphicalLasso(Xtrain, Xtest, alpha=0.1)
# res = es.fit_GraphicalLasso_CV(Xtrain, Xtest, n_jobs=None)
# res = es.fit_Shrinkage(Xtrain, Xtest, shrinkage=0.1)
# res = es.fit_Shrinkage_CV(Xtrain, Xtest, shrinkages=None,  cv_scoring="likelihood", n_jobs=None)
# res = es.fit_RIE(Xtrain, Xtest)
# res = es.fit_RIE_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_GradientAscent(Xtrain, Xtest, bootstrapping=False, stop='completion', val_frac=0.1)
# res = es.fit_GradientAscentWishart(Xtrain, Xtest, bootstrapping=False, stop='completion', val_frac=0.1)

C = X.T @ X / nsamples # empirical covariance
Cclean = res["Cclean"] # cleaned covariance


In [2]:
#load data
datadic=ourutils.getdata(paths[-1])
T,N=np.shape(datadic['032302'])





In [5]:
np.shape([el for el in datadic.values()]) 

(136, 652, 183)

In [ ]:
resfile = 'all_results_BIO.pickle'

train_fraction = 0.8
results={}

for i,( key, X) in enumerate(datadic.items()):
    if i<3:
        print (i)
        # Compute the corr mat C obtained flattening the training data from all patients
        Cbar = computeCbar([el for el in datadic.values()], train_fraction=train_fraction)
        Xtrain, Xtest = split_train_test(X, train_fraction=train_fraction, standardize=True, seed=i)
        key,value=run.single_run(i, Xtrain, Xtest, Cbar=Cbar)
        results[key]=value

0
1
2


In [13]:
for key, val in results[0].items():
    print (key,val['test_likelihood'])

PCA_CV_likelihood 174.50202507345483
Shrink_CV_likelihood 177.2986389916092
RIE_CV_likelihood 177.6852440017813
